In [ ]:
!git clone https://github.com/TrustAI/DeepConcolic.git

Cloning into 'DeepConcolic'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5124 (delta 28), reused 40 (delta 21), pack-reused 5070
Receiving objects: 100% (5124/5124), 263.83 MiB | 19.06 MiB/s, done.
Resolving deltas: 100% (2004/2004), done.
Checking out files: 100% (3825/3825), done.


In [ ]:
!ls

DeepConcolic  sample_data


In [ ]:
cd DeepConcolic/src/

/content/DeepConcolic/src


In [ ]:
ls

ap_lp.py              gen_har.py      nc.py             scripting.py
ap_main.py            gen_mnist.py    norms.py          ssc.py
bounds.py             l0_encoding.py  outs/             training.py
datasets.py           labels.txt      ploting.py        training_utils.py
deepconcolic_fuzz.py  lp.py           pulp_encoding.py  utils.py
deepconcolic.py       mcdc_pulp.py    pulp_norms.py     variables.py
engine.py             mcdc.py         __pycache__/      vgg16_example.py
filters.py            nc_l0.py        run_ssc.py
gen_fashion_mnist.py  nc_pulp.py      run_template.py*


In [ ]:
!sudo apt-get install python3.6
!pip install pulp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.6 is already the newest version (3.6.9-1~18.04ubuntu1.3).
python3.6 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
     |████████████████████████████████| 40.6MB 110kB/s 


# MNIST dataset

In [ ]:
!python deepconcolic.py --model ../saved_models/mnist_complicated.h5 --dataset a1 --max-iterations 20 --criterion ssclp --outputs outs/ 

2020-11-05 09:03:24.289092: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow version: 2.3.0
usage: deepconcolic.py [-h] [--model MODEL] [--inputs DIR] --outputs DIR
                       [--criterion nc, ssc...] [--setup-only] [--init INT]
                       [--max-iterations INT] [--save-all-tests]
                       [--rng-seed SEED] [--labels FILE]
                       [--dataset {mnist,fashion_mnist,cifar10,OpenML:har}]
                       [--vgg16-model] [--filters {LOF}] [--norm linf, l0]
                       [--input-rows INT] [--input-cols INT]
                       [--input-channels INT] [--cond-ratio FLOAT]
                       [--top-classes INT] [--layers LAYER [LAYER ...]]
                       [--feature-index INT] [--fuzzing] [--num-tests INT]
                       [--num-processes INT] [--sleep-time INT]
deepconcolic.py: error: argument --dataset: invalid choice: '

In [ ]:
# !python deepconcolic.py --model ../saved_models/mnist_complicated.h5 --dataset mnist --outputs outs/ --max-iterations 100 --criterion nc
iters = 25
comm = 'python deepconcolic.py --model ../saved_models/mnist_complicated.h5 --dataset mnist --outputs outs/ --max-iterations '+ str(iters)+' --criterion nc'
import os
cli_comm = ''+comm
exec('os.system(cli_comm)')
print(cli_comm)

python deepconcolic.py --model ../saved_models/mnist_complicated.h5 --dataset mnist --outputs outs/ --max-iterations 25 --criterion nc


CIFAR-10 dataset 

In [ ]:
!python deepconcolic.py --model ../saved_models/cifar10_complicated.h5 --dataset cifar10 --outputs outs/  --max-iterations 1000

2020-10-19 04:34:56.819729: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow version: 2.3.0
Loading cifar10 dataset... done.
2020-10-19 04:34:59.606198: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-19 04:34:59.616508: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-19 04:34:59.616561: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (df7b2cdee595): /proc/driver/nvidia/version does not exist
2020-10-19 04:34:59.622412: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200000000 Hz
2020-10-19 04:34:59.622838: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xdb42840 initialized for platform Host (this does not guarantee that XLA wil

# **custom dataset testing**

dependency pip install -q pyyaml h5py

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

In [6]:
data = pd.read_csv('1n.csv')
data.shape

(2859, 301)

In [5]:
data.describe()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,...,0.259,0.260,0.261,0.262,0.263,0.264,0.265,0.266,0.267,0.268,0.269,0.270,0.271,0.272,0.273,0.274,0.275,0.276,0.277,0.278,0.279,0.280,0.281,0.282,0.283,0.284,0.285,0.286,0.287,0.288,0.289,0.290,0.291,0.292,0.293,0.35114,0.294,0.295,0.296,2
count,2859.000000,2859.000000,2859.0,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.00000,2859.0,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.00000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,...,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000
mean,0.002498,0.003284,0.0,0.001442,0.000760,0.002182,0.002310,0.001235,0.002871,0.001309,0.001657,0.00181,0.0,0.001622,0.001553,0.001245,0.001529,0.001564,0.002152,0.000621,0.000784,0.001812,0.001361,0.00157,0.002266,0.001146,0.002121,0.002359,0.002546,0.001172,0.001080,0.002333,0.001548,0.001513,0.002193,0.000956,0.001738,0.001107,0.001929,0.002827,...,0.005456,0.003811,0.008150,0.006109,0.006452,0.006166,0.012005,0.005895,0.001353,0.007131,0.007629,0.006015,0.006021,0.007403,0.010118,0.007574,0.009565,0.010706,0.011293,0.006091,0.008450,0.007642,0.007799,0.004770,0.002151,0.010881,0.011771,0.014443,0.012765,0.011245,0.010337,0.014221,0.015693,0.014164,0.024065,0.017848,0.009044,0.015616,0.033927,4.570829
std,0.031047,0.040854,0.0,0.023607,0.017218,0.030853,0.031588,0.022616,0.035175,0.022358,0.027353,0.02740,0.0,0.023315,0.026572,0.024412,0.025006,0.024874,0.029575,0.014939,0.017763,0.026104,0.023477,0.02405,0.029101,0.020249,0.029053,0.030771,0.035445,0.022517,0.020598,0.029520,0.024176,0.022873,0.029605,0.020757,0.026013,0.020016,0.027144,0.034590,...,0.045276,0.036054,0.050209,0.043883,0.046656,0.045584,0.063318,0.042321,0.020239,0.046917,0.048534,0.044431,0.043520,0.048095,0.059011,0.050508,0.058910,0.060677,0.060864,0.042363,0.052364,0.050315,0.051428,0.038196,0.025850,0.057414,0.057372,0.074024,0.061305,0.058056,0.053246,0.063646,0.063712,0.067480,0.082356,0.072218,0.049087,0.062071,0.090905,0.969516
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [9]:
from sklearn.model_selection import train_test_split
X = data.iloc[:,:-1]
Y = data.iloc[:,300]

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.4,random_state=34)
scaler = preprocessing.StandardScaler().fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

array([[-0.07482649, -0.08452031,  0.        , ..., -0.18288902,
        -0.24548245,  4.76078377],
       [-0.07482649, -0.08452031,  0.        , ..., -0.18288902,
        -0.24548245,  2.46321717],
       [-0.07482649, -0.08452031,  0.        , ..., -0.18288902,
         4.31084773, -0.37270256],
       ...,
       [-0.07482649, -0.08452031,  0.        , ..., -0.18288902,
        -0.24548245, -0.37270256],
       [-0.07482649, -0.08452031,  0.        , ..., -0.18288902,
        -0.24548245, -0.37270256],
       [-0.07482649, -0.08452031,  0.        , ..., -0.18288902,
        -0.24548245,  2.12785876]])

In [ ]:
!pwd
!ls
# !cat Model11.h5
with open('Model11.h5',encoding='utf8') as f:
  for line in f.readlines():
    print(line)

In [ ]:
mdel = keras.models.load_model('Model11.h5')

ValueError: ignored

In [16]:
 model = Sequential([Dense(32,activation='relu',input_shape=(300,))
 ,Dense(8,activation='relu')
 ,Dense(7,activation='softmax')])

In [17]:
model.compile(optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [19]:
model.fit(X_train,to_categorical(Y_train),epochs=25, batch_size=32)

Epoch 1/25
54/54 [==============================] - 0s 1ms/step - loss: 0.7629 - accuracy: 0.7691
Epoch 2/25
54/54 [==============================] - 0s 1ms/step - loss: 0.7494 - accuracy: 0.7685
Epoch 3/25
54/54 [==============================] - 0s 1ms/step - loss: 0.7348 - accuracy: 0.7703
Epoch 4/25
54/54 [==============================] - 0s 1ms/step - loss: 0.7198 - accuracy: 0.7714
Epoch 5/25
54/54 [==============================] - 0s 1ms/step - loss: 0.7061 - accuracy: 0.7749
Epoch 6/25
54/54 [==============================] - 0s 1ms/step - loss: 0.6903 - accuracy: 0.7802
Epoch 7/25
54/54 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.7837
Epoch 8/25
54/54 [==============================] - 0s 2ms/step - loss: 0.6607 - accuracy: 0.7866
Epoch 9/25
54/54 [==============================] - 0s 1ms/step - loss: 0.6478 - accuracy: 0.7883
Epoch 10/25
54/54 [==============================] - 0s 1ms/step - loss: 0.6317 - accuracy: 0.7948
Epoch 11/25
54/54 [

In [13]:
model.evaluate(X_test,to_categorical(Y_test),batch_size=10)

115/115 [==============================] - 0s 863us/step - loss: 2.1579 - accuracy: 0.6993


[2.1579320430755615, 0.6993007063865662]

In [ ]:
model.save('custom_new.h5')

In [ ]:
(X.shape[1])

300